In [1]:
import numpy as np
import matplotlib.pyplot as plt

We have Burgers' equation

$$
\frac{\partial u}{\partial t} + \frac{\partial}{\partial x}f(u) = 0.
$$

Here $f(u) = \frac{1}{2}u^2$. We have discretized along time, so we have $u$ at time $n$, ie $u^n$. We have also defined our mesh along integer nodes, so $u_i$. Note the above can be rewritten as

$$
\frac{du_i}{dt} = -\left[\frac{\partial f}{\partial x}\right]_{x = x_i}.
$$

Define a function $h$ such that

$$
f(x) = \frac{1}{\Delta x}\int_{x_{i-\frac{1}{2}}}^{x_{i+\frac{1}{2}}}{h(\xi)d\xi}.
$$

Inserting this into the above and applying the Fundamental Theorem of Calculus, we see

$$
\frac{du_i}{dt} = -\frac{1}{\Delta x}\left(h_{i+\frac{1}{2}} - h_{i-\frac{1}{2}}\right).
$$

We now need to approximate $h_{i+\frac{1}{2}}$, we will do so by interpolating the known points using Lagrange polynomials. Now, we could very well use a single stencil and interpolate using a single Lagrange polynomial, but what if there are discontinuities?

To keep these in check, we employ first a five-point superstencil $S^{5}$, itself composed of three substencils $\left\{S_0^3,S_1^3,S_2^3\right\}$. Were we to use ENO, we would construct second order Lagrange polynomials from each of these stencils, interpolating $h_{i+\frac{1}{2}}$ and then going with the stencil that showed divided difference of the least magnitude. The interpolating polynomials are

$$
    p^{(1)}_{3} = \frac{\left(x - x_{i-1}\right)\left(x - x_{i}\right)}{\left(x_{i-2} - x_{i-1}\right)\left(x_{i-2} - x_{i}\right)}u_{i-2} + \frac{\left(x - x_{i-2}\right)\left(x - x_{i}\right)}{\left(x_{i-1} - x_{i-2}\right)\left(x_{i-1} - x_{i}\right)}u_{i-1} + \frac{\left(x - x_{i-2}\right)\left(x - x_{i-1}\right)}{\left(x_{i} - x_{i-2}\right)\left(x_{i} - x_{i-1}\right)}u_{i}, \\
    p^{(2)}_{3} = \frac{\left(x - x_{i}\right)\left(x - x_{i+1}\right)}{\left(x_{i-1} - x_{i}\right)\left(x_{i-1} - x_{i+1}\right)}u_{i-1} + \frac{\left(x - x_{i-1}\right)\left(x - x_{i+1}\right)}{\left(x_{i} - x_{i-1}\right)\left(x_{i} - x_{i+1}\right)}u_{i} + \frac{\left(x - x_{i-1}\right)\left(x - x_{i}\right)}{\left(x_{i+1} - x_{i-1}\right)\left(x_{i+1} - x_{i}\right)}u_{i+1}, \\
    p^{(3)}_{3} = \frac{\left(x - x_{i+1}\right)\left(x - x_{i+2}\right)}{\left(x_{i} - x_{i+1}\right)\left(x_{i} - x_{i+2}\right)}u_{i} + \frac{\left(x - x_{i}\right)\left(x - x_{i+2}\right)}{\left(x_{i+1} - x_{i}\right)\left(x_{i+1} - x_{i+2}\right)}u_{i+1} + \frac{\left(x - x_{i}\right)\left(x - x_{i+1}\right)}{\left(x_{i+2} - x_{i}\right)\left(x_{i+2} - x_{i+1}\right)}u_{i+2}.
$$

Plugging in $x_{i+\frac{1}{2}}$, we see:

$$
    \begin{split}
    p^{(1)}_{3}\left(x_{i+\frac{1}{2}}\right) &= \frac{\left(x_{i+\frac{1}{2}} - x_{i-1}\right)\left(x_{i+\frac{1}{2}} - x_{i}\right)}{\left(x_{i-2} - x_{i-1}\right)\left(x_{i-2} - x_{i}\right)}u_{i-2} + \frac{\left(x_{i+\frac{1}{2}} - x_{i-2}\right)\left(x_{i+\frac{1}{2}} - x_{i}\right)}{\left(x_{i-1} - x_{i-2}\right)\left(x_{i-1} - x_{i}\right)}u_{i-1} + \frac{\left(x_{i+\frac{1}{2}} - x_{i-2}\right)\left(x_{i+\frac{1}{2}} - x_{i-1}\right)}{\left(x_{i} - x_{i-2}\right)\left(x_{i} - x_{i-1}\right)}u_{i} \\
    &= \frac{\left(\frac{3}{2}\Delta x\right)\left(\frac{1}{2} \Delta x\right)}{\left(-\Delta x\right)\left(-2\Delta x\right)}u_{i-2} + \frac{\left(\frac{5}{2}\Delta x\right)\left(\frac{1}{2}\Delta x\right)}{\left(\Delta x\right)\left(-\Delta x\right)}u_{i-1} + \frac{\left(\frac{5}{2}\Delta x\right)\left(\frac{3}{2}\Delta x\right)}{\left(2\Delta x\right)\left(\Delta x\right)}u_{i} \\
    &= \frac{\left(\frac{3}{2}\right)\left(\frac{1}{2}\right)}{\left(-1\right)\left(-2\right)}u_{i-2} + \frac{\left(\frac{5}{2}\right)\left(\frac{1}{2}\right)}{\left(1\right)\left(-1\right)}u_{i-1} + \frac{\left(\frac{5}{2}\right)\left(\frac{3}{2}\right)}{\left(2\right)\left(1\right)}u_{i} \\
    &= \frac{3}{8}u_{i-2} - \frac{5}{4}u_{i-1} + \frac{15}{8}u_{i}
    \end{split} \\
    \begin{split}
    p^{(2)}_{3}\left(x_{i+\frac{1}{2}}\right) &= \frac{\left(x_{i+\frac{1}{2}} - x_{i}\right)\left(x_{i+\frac{1}{2}} - x_{i+1}\right)}{\left(x_{i-1} - x_{i}\right)\left(x_{i-1} - x_{i+1}\right)}u_{i-1} + \frac{\left(x_{i+\frac{1}{2}} - x_{i-1}\right)\left(x_{i+\frac{1}{2}} - x_{i+1}\right)}{\left(x_{i} - x_{i-1}\right)\left(x_{i} - x_{i+1}\right)}u_{i} + \frac{\left(x_{i+\frac{1}{2}} - x_{i-1}\right)\left(x_{i+\frac{1}{2}} - x_{i}\right)}{\left(x_{i+1} - x_{i-1}\right)\left(x_{i+1} - x_{i}\right)}u_{i+1}, \\
    &= \frac{\left(\frac{1}{2}\Delta x\right)\left(-\frac{1}{2}\Delta x\right)}{\left(-\Delta x\right)\left(-2\Delta x\right)}u_{i-1} + \frac{\left(\frac{3}{2}\Delta x\right)\left(-\frac{1}{2}\Delta x\right)}{\left(\Delta x\right)\left(-\Delta x\right)}u_{i} + \frac{\left(\frac{3}{2}\Delta x\right)\left(\frac{1}{2}\Delta x\right)}{\left(2\Delta x\right)\left(\Delta x\right)}u_{i+1}, \\
    &= \frac{\left(\frac{1}{2}\right)\left(-\frac{1}{2}\right)}{\left(-1\right)\left(-2\right)}u_{i-1} + \frac{\left(\frac{3}{2}\right)\left(-\frac{1}{2}\right)}{\left(1\right)\left(-1\right)}u_{i} + \frac{\left(\frac{3}{2}\right)\left(\frac{1}{2}\right)}{\left(2\right)\left(1\right)}u_{i+1}, \\
    &= -\frac{1}{8}u_{i-1} + \frac{3}{4}u_{i} + \frac{3}{8}u_{i+1}, \\
    \end{split} \\
    \begin{split}
    p^{(3)}_{3}\left(x_{i+\frac{1}{2}}\right) &= \frac{\left(x_{i+\frac{1}{2}} - x_{i+1}\right)\left(x_{i+\frac{1}{2}} - x_{i+2}\right)}{\left(x_{i} - x_{i+1}\right)\left(x_{i} - x_{i+2}\right)}u_{i} + \frac{\left(x_{i+\frac{1}{2}} - x_{i}\right)\left(x_{i+\frac{1}{2}} - x_{i+2}\right)}{\left(x_{i+1} - x_{i}\right)\left(x_{i+1} - x_{i+2}\right)}u_{i+1} + \frac{\left(x_{i+\frac{1}{2}} - x_{i}\right)\left(x_{i+\frac{1}{2}} - x_{i+1}\right)}{\left(x_{i+2} - x_{i}\right)\left(x_{i+2} - x_{i+1}\right)}u_{i+2} \\
    &= \frac{\left(-\frac{1}{2}\Delta x\right)\left(-\frac{3}{2}\Delta x\right)}{\left(-\Delta x\right)\left(-2\Delta x\right)}u_{i} + \frac{\left(\frac{1}{2}\Delta x\right)\left(-\frac{3}{2}\Delta x\right)}{\left(\Delta x\right)\left(-\Delta x\right)}u_{i+1} + \frac{\left(\frac{1}{2}\Delta x\right)\left(-\frac{1}{2}\Delta x\right)}{\left(2\Delta x\right)\left(\Delta x\right)}u_{i+2} \\
    &= \frac{\left(-\frac{1}{2}\right)\left(-\frac{3}{2}\right)}{\left(-1\right)\left(-2\right)}u_{i} + \frac{\left(\frac{1}{2}\right)\left(-\frac{3}{2}\right)}{\left(1\right)\left(-1\right)}u_{i+1} + \frac{\left(\frac{1}{2}\right)\left(-\frac{1}{2}\right)}{\left(2\right)\left(1\right)}u_{i+2} \\
    &= \frac{3}{8}u_{i} + \frac{3}{4}u_{i+1} - \frac{1}{8}u_{i+2} \\
    \end{split}
$$

Now, were we to interpolate the value using the five point stencil, we see

$$
u_{i+\frac{1}{2}} = \frac{1}{30}u_{i-2} - \frac{13}{60}u_{i-1} + \frac{47}{60}u_{i} + \frac{9}{20}u_{i+1} - \frac{1}{20}u_{i+2}.
$$

Were we to combine this with the three point stencils that give us $u_{i+\frac{1}{2}}^{(1)}, u_{i+\frac{1}{2}}^{(2)}, u_{i+\frac{1}{2}}^{(3)}$, we would get the following

$$
\begin{split}
u_{i+\frac{1}{2}} &= \gamma_1u_{i+\frac{1}{2}}^{(1)} + \gamma_2u_{i+\frac{1}{2}}^{(2)} + \gamma_3u_{i+\frac{1}{2}}^{(3)} \\
&= \gamma_1\left(\frac{3}{8}u_{i-2} - \frac{5}{4}u_{i-1} + \frac{15}{8}u_{i}\right) + \gamma_2\left(-\frac{1}{8}u_{i-1} + \frac{3}{4}u_{i} + \frac{3}{8}u_{i+1}\right) + \gamma_3\left(\frac{3}{8}u_{i} + \frac{3}{4}u_{i+1} - \frac{1}{8}u_{i+2}\right) \\
&= \frac{1}{30}u_{i-2} - \frac{13}{60}u_{i-1} + \frac{47}{60}u_{i} + \frac{9}{20}u_{i+1} - \frac{1}{20}u_{i+2}
\end{split}
$$

We immediately see $\gamma_1 = \frac{1}{30}\frac{8}{3} = \frac{8}{90}$. Likewise, $\gamma_3 = \frac{8}{20} = \frac{2}{5}$. Using these two values, we compute the last

$$
\gamma_1\frac{5}{4}u_{i-1} - \gamma_2\frac{1}{8}u_{i-1} = -\frac{13}{60}u_{i-1} \implies \gamma_2 = -8\left(-\frac{13}{60}-\gamma_1\frac{5}{4}\right) = \frac{118}{45}
$$

This is a linear system

$$
\begin{pmatrix}
\end{pmatrix}

\begin{pmatrix}
\end{pmatrix}
=
\begin{pmatrix}
\end{pmatrix}
$$